## Overview
<a href="https://archive.ics.uci.edu/ml/datasets/online+retail">Online retail</a> is a transnational dataset which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail. The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

## Source
UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/online+retail

## Business Goal
To segment the Customers based on RFM so that the company can target its customers efficiently.

## Methodology

1. [Reading and Understanding the Data](#1) <br>
    a. Creating a Data Dictionary
2. [Data Cleaning](#2)
3. [Data Preparation](#3) <br>
    a. Scaling Variables
4. [Model Building](#4) <br>
    a. K-means Clustering <br>
    b. Finding the Optimal K
5. [Final Analysis](#5)

<a id="1"></a> <br>
### 1 : Reading and Understanding Data

In [ ]:
# import required libraries for dataframe and visualization

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import plotly as py 
import plotly.graph_objs as go

from plotly.offline import init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

# import required libraries for clustering
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree

In [ ]:
# Reading the data on which analysis needs to be done
retail = pd.read_csv('OnlineRetail.csv', encoding='ISO-8859-1')

#### Data Dictionary 

First Header  | Definition    |  Description  | Data Type
------------- | ------------- | ------------- | -------------
InvoiceNo  | Invoice number | A 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation. | Nominal
StockCode | Product (item) code | A 5-digit integral number uniquely assigned to each distinct product. | Nominal
Description | Product (item) name | Name of Product | Nominal
Quantity | Quantity | The quantities of each product (item) per transaction | Numeric
InvoiceDate | Invoice Date and time | The day and time when each transaction was generated. | Numeric
UnitPrice | Unit price | Product price per unit in sterling. | Numeric
CustomerID | Customer number | A 5-digit integral number uniquely assigned to each customer. | Nominal
Country | Country name | The name of the country where each customer resides. | Nominal

In [ ]:
#show all columns on output
pd.set_option('display.max_columns', None)

In [ ]:
retail.head(10)

`Explanation:` First, the dataset is loaded into Jupyter notebook, and the initial step was to show the first ten rows of the dataset.

In [ ]:
# shape of df
retail.shape

In [ ]:
# df description
retail.describe()

In [ ]:
retail.info()

<a id="2"></a> <br>
### 2 : Data Cleaning

In [ ]:
# Inspecting InvoiceDate

retail['InvoiceDate'].head(10)

In [ ]:
# Converting InvoiceDate to datetime

retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate'], dayfirst=True)

In [ ]:
# Inspecting InvoiceDate

retail['InvoiceDate'].head(10)

In [ ]:
retail.info()

In [ ]:
# Calculating the Missing Values % contribution in DF
df_null = round(100*(retail.isnull().sum())/len(retail), 2)
df_null

In [ ]:
# Plotting the df_null

plt.figure(figsize=(16,8))
sns.barplot(x=df_null.index, y=df_null.values, alpha=0.8)
plt.title('Missing Values (Pre Cleaning)')
plt.ylabel('Missing Values %')
plt.xlabel('Columns')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Droping rows having missing values
retail = retail.dropna()
retail.shape

`Explanation:` The missing values were scanned then it was removed.

In [ ]:
# Calculating the Missing Values % contribution in DF
retail.isna().sum()

In [ ]:
# Plotting a barplot for the retail to check if null values still exist

plt.figure(figsize=(16,8))
sns.barplot(x=retail.columns, y=retail.isna().sum().values, alpha=0.8)
plt.title('Missing Values (Post Cleaning)')
plt.ylabel('Missing Values')
plt.xlabel('Columns')
plt.xticks(rotation=90)
plt.show()

`Explanation:` The bar chart was represented by computing the percentage of missing values by column before and after preprocessing

Most of the missing values are in the ‘CustomerID’ column, and few are in the ‘Description’ column.

In [ ]:
# Changing the datatype of Customer Id as per Business understanding
retail['CustomerID'] = retail['CustomerID'].astype(str)
retail.info()

<a id="3"></a> <br>
### 3 : Data Preparation

#### Customers will be analyzed based on 3 factors:
- R (Recency): Number of days since last purchase
- F (Frequency): Number of tracsactions
- M (Monetary): Total amount of transactions (revenue contributed)

In [ ]:
# New Attribute : Monetary
retail['Amount'] = retail['Quantity'] * retail['UnitPrice']
rfm_m = retail.groupby('CustomerID')['Amount'].sum()
rfm_m = rfm_m.reset_index()
rfm_m.head(5)

In [ ]:
# New Attribute : Frequency
rfm_f = retail.groupby('CustomerID')['InvoiceNo'].count()
rfm_f = rfm_f.reset_index()
rfm_f.columns = ['CustomerID', 'Frequency']
rfm_f.head(5)



In [ ]:
# Merging the two dfs
rfm = pd.merge(rfm_m, rfm_f, on='CustomerID', how='inner')
print(rfm.shape)
rfm.head(5)

In [ ]:
# New Attribute : Recency
# Convert to datetime to proper datatype
retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate'],format='%d-%m-%Y %H:%M')



In [ ]:
# Compute the maximum date to know the last transaction date
max_date = max(retail['InvoiceDate'])
max_date

In [ ]:
# Compute the difference between max date and transaction date
retail['Diff'] = max_date - retail['InvoiceDate']
retail.head(5)

In [ ]:
# Compute last transaction date to get the recency of customers
rfm_p = retail.groupby('CustomerID')['Diff'].min()
rfm_p = rfm_p.reset_index()
rfm_p.head(5)

In [ ]:
# Extract number of days only
rfm_p['Diff'] = rfm_p['Diff'].dt.days
rfm_p.head(5)


In [ ]:
# Merge tha dataframes to get the final RFM dataframe
rfm = pd.merge(rfm, rfm_p , on='CustomerID', how='inner')

In [ ]:
rfm.columns = ['CustomerID', 'Amount', 'Frequency', 'Recency']
rfm.head(5)

## EDA

In [ ]:
# Boxplot to identify outliers in the dataset

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.boxplot(data=rfm['Amount'], color='skyblue')
plt.title('Amount Boxplot')
plt.xlabel('Amount')

plt.subplot(1, 3, 2)
sns.boxplot(data=rfm['Frequency'], color='lightgreen')
plt.title('Frequency Boxplot')
plt.xlabel('Frequency')

plt.subplot(1, 3, 3)
sns.boxplot(data=rfm['Recency'], color='salmon')
plt.title('Recency Boxplot')
plt.xlabel('Recency')

plt.tight_layout()
plt.show()

`Explanation:` 

Boxplot on the other hand, helped us with summary statistics like easily identifying the median, quartile, and most especially, it is great for spotting outliers and understanding the data spread.

Similarly, like the histogram, it showed potential outliers distributed by identifying points that are isolated and far from the point clusters.


In [ ]:
# Histogram for the distribution of at least two key numerical features

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.histplot(rfm['Amount'], color='skyblue', kde=True)
plt.title('Amount Distribution')
plt.xlabel('Amount')

plt.subplot(1, 3, 2)
sns.histplot(rfm['Frequency'], color='lightgreen', kde=True)
plt.title('Frequency Distribution')
plt.xlabel('Frequency')

plt.subplot(1, 3, 3)
sns.histplot(rfm['Recency'], color='salmon', kde=True)
plt.title('Recency Distribution')
plt.xlabel('Recency')

plt.tight_layout()
plt.show()

`Explanation:`

Amount and Frequency both show a highly skewed distribution with numerous outliers, suggesting that a few customers make very large transactions or purchase very frequently, in contrast to the majority.

Recency is less skewed but still shows variability, with most customers being more recent in their activity, but some haven’t made purchases for a long time.

The histogram focuses on the distribution of a single variable, and we could understand the data through the skewness, normality, or multimodality.


In [ ]:
# Correlation heatmap to show relationships between numerical variables

plt.figure(figsize=(10,5))
sns.heatmap(rfm.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

`Explanation:`

The correlation heatmap helped us in which features or variables have a strong relationship or correlation with each other. When reviewing the correlation heatmap it gives

We found a positive correlation with the Amount and Frequency that we interpret as those customers who purchase more, spends more money.

Low correlation of CustomerID shows that it is trivial or it doesn’t correlate closely with features such as Amount, Frequency, and Recency.

#### Rescaling the Attributes

It is extremely important to rescale the variables so that they have a comparable scale.<br>
There are two common ways of rescaling:

1. Min-Max scaling 
2. Standardization (mean-0, sigma-1) 

Here we execute Standard Scaling.

In [ ]:
# Density Plot Before Scaling
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.kdeplot(rfm['Amount'], color='skyblue', fill=True)
plt.title('Amount Distribution')
plt.xlabel('Amount')

plt.subplot(1, 3, 2)
sns.kdeplot(rfm['Frequency'], color='lightgreen', fill=True)
plt.title('Frequency Distribution')
plt.xlabel('Frequency')

plt.subplot(1, 3, 3)
sns.kdeplot(rfm['Recency'], color='salmon', fill=True)
plt.title('Recency Distribution')

plt.tight_layout()
plt.show()

`Explanation:`

Before Scaling: The data for Amount, Frequency, and Recency are on vastly different scales.

 For example, Amount is spread across thousands, while Recency could span hundreds of days, and Frequency ranges from single digits to a few hundred.


In [ ]:
# Rescaling the Attributes
rfm_df = rfm[['Amount', 'Frequency', 'Recency']]

# Instantiate
scaler = StandardScaler()

# fit_transform
rfm_df_scaled = scaler.fit_transform(rfm_df)
rfm_df_scaled = pd.DataFrame(rfm_df_scaled)
rfm_df_scaled.shape

rfm_df_scaled.columns = ['Amount', 'Frequency', 'Recency']

In [ ]:
# Density Plot After Scaling
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.kdeplot(rfm_df_scaled['Amount'], color='skyblue', fill=True)
plt.title('Amount Distribution')
plt.xlabel('Amount')

plt.subplot(1, 3, 2)
sns.kdeplot(rfm_df_scaled['Frequency'], color='lightgreen', fill=True)
plt.title('Frequency Distribution')
plt.xlabel('Frequency')

plt.subplot(1, 3, 3)
sns.kdeplot(rfm_df_scaled['Recency'], color='salmon', fill=True)
plt.title('Recency Distribution')

plt.tight_layout()
plt.show()

`Explanation:`

After Standard Scaler: After applying the Standard Scaler (mean of 0, standard deviation of 1), all variables are transformed to a standard distribution. 

This ensures that they have comparable scales while maintaining the differences between high and low values.


## <span style="color: red;">Execute MinMax Scaling in the next box</span> 

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Rescaling the Attributes
rfm_df = rfm[['Amount', 'Frequency', 'Recency']]

# Instantiate 
scaler = MinMaxScaler()

# fit_transform
rfm_df_scaled = scaler.fit_transform(rfm_df)
rfm_df_scaled = pd.DataFrame(rfm_df_scaled)
rfm_df_scaled.shape

rfm_df_scaled.columns = ['Amount', 'Frequency', 'Recency']


In [ ]:
# Density Plot After Scaling
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.kdeplot(rfm_df_scaled['Amount'], color='skyblue', fill=True)
plt.title('Amount Distribution')
plt.xlabel('Amount')

plt.subplot(1, 3, 2)
sns.kdeplot(rfm_df_scaled['Frequency'], color='lightgreen', fill=True)
plt.title('Frequency Distribution')
plt.xlabel('Frequency')

plt.subplot(1, 3, 3)
sns.kdeplot(rfm_df_scaled['Recency'], color='salmon', fill=True)
plt.title('Recency Distribution')

plt.tight_layout()
plt.show()

`After MinMax Scaling`: MinMax scaling brings all the values into the range [0, 1], which is helpful for models sensitive to the magnitude of input values, especially clustering algorithms like K-means.


In [ ]:
rfm_df_scaled.head(5)

`Explain why this transformation is necessary for the next steps of the analysis: 
`

Clustering Algorithms (like K-means) are highly sensitive to the scale of the data. Variables on different scales can dominate the distance calculations, leading to biased clusters.

Standard Scaling helps preserve the original distribution but makes variables comparable, which is ideal for distance-based algorithms.

MinMax Scaling is crucial when the range of variables is important for algorithm stability, ensuring no variable dominates due to large ranges.


<a id="4"></a> <br>
### 4 : Building the Model

### K-Means Clustering

K-means clustering is one of the simplest and popular unsupervised machine learning algorithms.<br>

The algorithm works as follows:

- First we initialize k points, called means, randomly.
- We categorize each item to its closest mean and we update the mean’s coordinates, which are the averages of the items categorized in that mean so far.
- We repeat the process for a given number of iterations and at the end, we have our clusters.

In [ ]:
# k-means with some arbitrary k
Kmeans = KMeans(n_clusters=4, max_iter=50)
Kmeans.fit(rfm_df_scaled)

In [ ]:
Kmeans.labels_

In [ ]:
# Create a K-means function here
def K_Means(X, n):
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    model = KMeans(n)
    model.fit(X)
    cluster_labels = model.predict(X)
    cent = model.cluster_centers_
    return cluster_labels, cent

In [ ]:
clust_labels, cent = K_Means(rfm_df, 3)
kmeans = pd.DataFrame(clust_labels)
rfm_df.insert((rfm_df.shape[1]), 'kmeans', kmeans)


In [ ]:
rfm_df.head(5)

In [ ]:
# Plot your clusters
def Plot3dClustering(n, X, type_c):
    data = []
    clusters = []
    colors = ['rgb(228,26,28)', 'rgb(55,126,184)', 'rgb(77,175,74)']
    
    for i in range(n):
        name = i
        color = colors[i]
        x = X[X[type_c] == i]['Amount']
        y = X[X[type_c] == i]['Frequency']
        z = X[X[type_c] == i]['Recency']
        
        trace = dict(
            name = name,
            x = x, y = y, z = z,
            type = 'scatter3d',
            mode = 'markers',
            marker = dict(size = 3, color = color, line = dict(width = 0)))
        data.append(trace)
        
        cluster = dict(
            color = color,
            opacity = 0.1,
            type = 'mesh3d',
            alphahull = 7,
            name = "y",
            x = x, y = y, z = z)
        data.append(cluster)
        
    layout = dict(
        width = 800,
        height = 550,
        autosize = False,
        title = '3D Clustering Plot',
        scene = dict(
            xaxis = dict(
                gridcolor = 'rgb(255, 255, 255)',
                zerolinecolor = 'rgb(255, 255, 255)',
                showbackground = True,
                title='Amount',
                backgroundcolor = 'rgb(230, 230, 230)',
                ),
            yaxis = dict(
                gridcolor = 'rgb(255, 255, 255)',
                zerolinecolor = 'rgb(255, 255, 255)',
                showbackground = True,
                title='Frequency',
                backgroundcolor = 'rgb(230, 230, 230)',
                ),
            zaxis = dict(
                gridcolor = 'rgb(255, 255, 255)',
                zerolinecolor = 'rgb(255, 255, 255)',
                showbackground = True,
                title='Recency',
                backgroundcolor = 'rgb(230, 230, 230)',
                ),
            aspectratio = dict(x=1, y=1, z=0.7),
            aspectmode = 'manual'
        ),
    )
        
    fig = dict(data = data, layout = layout)
    iplot(fig, filename='3d-scatter-colorscale', validate=False)
    

In [ ]:
# Plot3dClustering(n=3, X=rfm_df, type_c='kmeans')

`Describe the characteristics of the 3D Scatter Plot
`

The data points are mainly differentiated by Recency, with clear divisions between recent, moderately recent, and older transactions.

Amount and Frequency also contribute to cluster differentiation, especially for older, high-amount transactions (green) versus more recent, low-amount transactions (blue and red).


`Describe the characteristics of each cluster and the clustering results based on the visualization.
`

Red Cluster (central, vertical stretch): Represents transactions or events with varying Recency but generally low Frequency. The Amount stays moderate, ranging from 0 to 100k. These could be regular but infrequent transactions happening over a wide range of time periods.

Green Cluster (spread at the base): Represents older transactions (low Recency), with a broad range in Frequency and Amount. Some transactions in this group have high amounts (up to 250k), suggesting less frequent but larger, older transactions.

Blue Cluster (top, stacked): Represents recent transactions with low Frequency and smaller Amount values (up to 50k). These are more recent events but occur less frequently and involve smaller monetary amounts.


In [ ]:
# Cluster Centroid Plot

plt.figure(figsize=(10, 5))
plt.title('Centroid Plot')
plt.xlabel('Amount')
plt.ylabel('Frequency')
plt.scatter(rfm_df['Amount'], rfm_df['Frequency'], c=rfm_df['kmeans'], cmap='rainbow')
plt.scatter(cent[:,0], cent[:,1], c='black', s=300, alpha=0.5)
plt.show()

## <span style="color: red;">Finding the Optimal Number of Clusters</span> 

#### Elbow Curve to get the right number of Clusters
A fundamental step for any unsupervised algorithm is to determine the optimal number of clusters into which the data may be clustered. The Elbow Method is one of the most popular methods to determine this optimal value of k.

In [ ]:
X = rfm_df
wcss = []
for i in range(1,8):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=7, random_state=0)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
plt.plot(range(1,8), wcss)
plt.title('Elbow Method')

In [ ]:
from sklearn.metrics import silhouette_score

# Visualize silhouette score for each cluster using a Silhouette plot
silhouette_avg = silhouette_score(rfm_df_scaled, rfm_df['kmeans'])
print(f'Silhouette Score for 3 clusters: {silhouette_avg}')

import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_samples

# Calculate silhouette values
sample_silhouette_values = silhouette_samples(rfm_df_scaled, rfm_df['kmeans'])

# Create a silhouette plot
plt.figure(figsize=(10, 6))
y_lower = 10
for i in range(3):  # Change to 3 for 3 clusters
    # Aggregate the silhouette scores for samples belonging to the current cluster
    ith_cluster_silhouette_values = sample_silhouette_values[rfm_df['kmeans'] == i]
    ith_cluster_silhouette_values.sort()
    
    # Get the size of the cluster
    size_cluster_i = ith_cluster_silhouette_values.shape[0]
    
    # Compute the y position for the current cluster
    y_upper = y_lower + size_cluster_i
    
    # Fill the silhouette plot for the current cluster
    plt.fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_silhouette_values)
    
    # Label the silhouette plots
    plt.text(-0.05, y_lower + 0.5 * size_cluster_i, f'Cluster {i + 1}')
    
    # Update the next y_lower position
    y_lower = y_upper + 10  # 10 for the 10px separation between clusters

plt.title('Silhouette Plot for the 3 Clusters')
plt.xlabel('Silhouette Coefficient Values')
plt.ylabel('Cluster Label')
plt.axvline(x=silhouette_avg, color='red', linestyle='--')
plt.show()

`Explanation:`The silhouette plot helps determine whether clusters are well-defined and can reveal points that may be misclassified or potential outliers. By examining the silhouette coefficient values, we can assess how well each point fits into its assigned cluster. Points with high values (close to 1) are well-clustered, while negative or near-zero values suggest poor clustering, misclassification, or the presence of outliers.

The plot revealed that:

Overall, the silhouette score across all clusters is relatively high, signifying that the clustering is good overall and points are well-placed.

However, a few points show low or negative silhouette values, particularly in Cluster 1, which may indicate potential outliers or points that are not well separated from other clusters. These points might benefit from further investigation or reassignment to other clusters, as they do not fit as neatly into the current grouping.

In summary, the clustering is robust, but there are some instances of borderline points or possible outliers that may require attention to refine the model further.


## <span style="color: red;">Box Plots of Clusters created</span> 

In [ ]:
# Creating the plot
plt.figure(figsize=(16, 10))

# Create a subplot for each feature
for i, feature in enumerate(['Amount', 'Frequency', 'Recency'], 1):
    plt.subplot(1, 3, i)
    sns.boxplot(x='kmeans', y=feature, data=rfm_df)
    plt.title(f'Boxplot of {feature} by Cluster')

# Show the plot
plt.tight_layout()
plt.show()


`Analysis:`

- The box plot reveals a significant disparity in the spending behavior across the clusters.

- Overall, the box plots highlight the distinct characteristics of each cluster in terms of spending habits, purchase frequency, and recency of transactions. Cluster 0 represents low spenders who shop infrequently but recently, while Cluster 1 consists of moderate spenders with a balanced shopping frequency. Cluster 2, on the other hand, captures high spenders who shop frequently but may have become less recent, indicating an opportunity for targeted marketing strategies to rekindle their engagement. This differentiation in customer behavior is essential for tailoring marketing efforts and improving customer relationship management strategies. ​


<a id="5"></a> <br>
## Step 5 : Final Analysis

## <span style="color: red;">Findings</span> 

In [ ]:
# Show the distribution of data points across the clusters using pie chart

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
rfm_df['kmeans'].value_counts().plot.pie(autopct='%1.1f%%')
plt.title('Cluster Distribution')
plt.ylabel('')
plt.show()

`Explanation:`

The pie chart shows the distribution of three customer clusters, with Cluster 0 accounting for 74.5%, Cluster 1 for 25.2%, and Cluster 2 for only 0.3%. These clusters are visually represented in the scatter plot, where Cluster 0 (blue) consists of customers who make frequent and recent purchases with high spending, dominating the plot. Cluster 1 (orange/red) includes occasional buyers with moderate frequency and recency, while Cluster 2 (green) represents a small group of inactive customers with low engagement, corresponding to its minimal share in the pie chart. The scatter plot thus reinforces the cluster sizes shown in the pie chart by depicting their purchasing behavior.

#### Student Name: 4CSC
- Benjamin Francis Abadila
- Angelo Dela Paz
- Carl Mitzchel Padua
- Edjin Jerney Payumo
- Levin Jacob Sta. Cruz